In [ ]:
!pip install import-ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as dr
import random as random
import copy
import import_ipynb
import sys
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import ndcg_score
# import libauc
# from libauc.losses.ranking import NDCG_Loss, ListwiseCE_Loss
# from libauc.optimizers import SONG


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%run /content/drive/MyDrive/FILNAL_PROJECT/utils_func.ipynb import *

In [ ]:
%run /content/drive/MyDrive/FILNAL_PROJECT/TDG_model.ipynb import *

2.0.1+cu118
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html


Import data

In [ ]:
path_data='/content/drive/MyDrive/FILNAL_PROJECT/sp500_stocks.csv'
path_compagnie='/content/drive/MyDrive/FILNAL_PROJECT/sp500_companies.csv'
res=load_stock_data(path_data,path_compagnie)
data_stock=build_feature(res[0],res[2])
stock_rank_res=stock_rank(data_stock,res[1],'NMS')
final_data_res=final_data(data_stock,res[1],'NMS',stock_rank_res)
spliten_data=split_datasets_for_training(final_data_res[0],final_data_res[1],60)


<ipython-input-22-b34e0010b515>:8: FutureWarning: Dropping of nuisance columns in DataFrame.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  rank_stock=pd.concat([stock_return["Date"].iloc[1:],stock_return.iloc[1:,:].rank(axis=1,method="min")],axis=1,join="inner")


In [ ]:
stock_rank_res[0].head(5)

,Date,AAL_rt,AAPL_rt,ABMD_rt,ACGL_rt,ADBE_rt,ADI_rt,ADP_rt,ADSK_rt,AEP_rt,...,VRSN_rt,VRTX_rt,VTRS_rt,WBA_rt,WDC_rt,WYNN_rt,XEL_rt,XRAY_rt,ZBRA_rt,ZION_rt
1,2010-01-05,127.0,72.0,6.0,52.0,101.0,56.0,43.0,14.0,20.0,...,92.0,3.0,23.0,34.0,116.0,124.0,18.0,17.0,55.0,121.0
2,2010-01-06,4.0,16.0,18.0,52.0,63.0,67.0,62.0,81.0,102.0,...,26.0,14.0,29.0,36.0,75.0,24.0,79.0,95.0,35.0,127.0
3,2010-01-07,119.0,62.0,70.0,44.0,11.0,35.0,69.0,92.0,97.0,...,20.0,23.0,15.0,93.0,10.0,115.0,51.0,104.0,4.0,127.0
4,2010-01-08,6.0,74.0,3.0,39.0,28.0,69.0,43.0,121.0,94.0,...,79.0,4.0,2.0,57.0,67.0,26.0,50.0,46.0,36.0,10.0
5,2010-01-11,5.0,21.0,6.0,50.0,13.0,37.0,85.0,67.0,105.0,...,52.0,124.0,117.0,75.0,73.0,49.0,103.0,110.0,80.0,93.0


In [ ]:
# silhouette','calinski_harabasz',distortion


In [ ]:
# proceed_cluster=build_cluster(metric="dtw",KElbowmetric='distortion', iteration=20, num_clus=8)
# predict_cluster=[]
# for i in range(len(spliten_data[4])):
#   predict_cluster.append(proceed_cluster.predict(spliten_data[4][i],k=(2,10),best=True))

# torch.save(predict_cluster, '/content/drive/MyDrive/FILNAL_PROJECT/cluster_distorsion_full2.pt')

In [ ]:
# predict_cluster

In [ ]:
# torch.save(predict_cluster, '/content/drive/MyDrive/FILNAL_PROJECT/cluster_distorsion_full.pt')

In [ ]:
tensor_list = torch.load('/content/drive/MyDrive/FILNAL_PROJECT/cluster_distorsion_full2.pt')

In [ ]:
x=spliten_data[0]
y=spliten_data[2]
market=res[1]
name="NMS"


In [ ]:
batch_x,batch_y,adj_x=batch_data(x,y,tensor_list,market,name,batch_size=2,seq_len=60)

In [ ]:
model=TGCNMODEL(in_channels=60,out_channels=50,hidden_channel=55,emb=26,num_nodes=127,num_layers=2, dropout=0.001)

In [ ]:

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.02)


In [ ]:
# criterion=nn.MSELoss() #try later

In [ ]:

num_epochs=100
best_loss=1
num_t=len(batch_x)-int(0.2*len(batch_x))

for epoch in range(num_epochs):
    train_loss=0
    test_loss=0

    for input_batch,target_batch,adj_batch in zip(batch_x[:num_t],batch_y[:num_t],adj_x[:num_t]) :

        optimizer.zero_grad()
        output_batch = model(input_batch,adj_batch)
        loss = (1-ndcg_score(target_batch,output_batch,k=5)).requires_grad_(True)
        # loss=criterion(output_batch.reshape(1,-1),target_batch.reshape(1,-1))

        # loss
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()
    train_loss=train_loss/num_t

    # EVALUTION ON TESTING DATA SET
    for input_test,target_test,adj_test in zip(batch_x[num_t:],batch_y[num_t:],adj_x[num_t:]):
          output_batch = model(input_test,adj_test)
          test_loss+= (1-ndcg_score(target_test,output_batch,k=5)).item()
          # test_loss+=criterion(output_batch.reshape(-1,1),target_test.reshape(-1,1))

    test_loss=test_loss/len(batch_x[num_t:])

    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Test Loss '+str(test_loss))


    if test_loss<best_loss:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_loss = test_loss





In [ ]:
best_loss

0.5500317982628814

1. TEST_LOSS =0.5596,BATCH_SIZE=2, DROUP_OUT=0.01,LR=0.001,out_channels=50,hidden_channel=55,top=10,metric=distorsion

2. TEST_LOSS =0.5586,BATCH_SIZE=2, DROUP_OUT=0.01,LR=0.01,out_channels=50,hidden_channel=55,,top=10,metric =distorsion

3. TEST_LOSS =0.5547,BATCH_SIZE=2, DROUP_OUT=0.01,LR=0.02,out_channels=50,hidden_channel=55,,top=10,metric =distorsion

3. TEST_LOSS =0.5425,BATCH_SIZE=2, DROUP_OUT=0.001,LR=0.02,out_channels=50,hidden_channel=55,,top=10,metric =distorsion

